In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

# Configuração do TensorFlow/Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

# Fixar seeds para reprodutibilidade
np.random.seed(42)
tf.random.set_seed(42)


def find_matching_files(imputed_folder, non_imputed_folder):
    """
    Encontra arquivos correspondentes entre as pastas com e sem imputação
    Retorna dicionário com {link: {'imputed': path, 'non_imputed': path}}
    """
    # Buscar arquivos imputados (formato: ac-am_stacking.csv)
    imputed_files = glob.glob(os.path.join(imputed_folder, "*_stacking.csv"))
    
    # Buscar arquivos não imputados (formato: ac-am_merged.csv)
    non_imputed_files = glob.glob(os.path.join(non_imputed_folder, "*_merged.csv"))
    
    # Criar mapeamento de links para arquivos não imputados
    non_imputed_dict = {}
    for file_path in non_imputed_files:
        filename = os.path.basename(file_path)
        link = filename.replace('_merged.csv', '')
        non_imputed_dict[link] = file_path
    
    # Combinar arquivos correspondentes
    matched_files = {}
    
    for imputed_file in imputed_files:
        filename = os.path.basename(imputed_file)
        link = filename.replace('_stacking.csv', '')
        
        # Verificar se existe arquivo correspondente sem imputação
        if link in non_imputed_dict:
            matched_files[link] = {
                'imputed': imputed_file,
                'non_imputed': non_imputed_dict[link]
            }
    
    print(f"Encontrados {len(matched_files)} pares de arquivos correspondentes")
    return matched_files


def prepare_univariate_sequences(data, n_steps, target_col='Vazao_BBR'):
    """
    Prepara sequências UNIVARIADAS para LSTM/GRU usando apenas Vazao_BBR
    A predição é feita 1 passo à frente (next timestep)
    """
    # Usar apenas a coluna Vazao_BBR
    vazao_data = data[[target_col]].values
    
    # Normalizar dados
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(vazao_data)
    
    X, y = [], []
    indices = []  # Para rastrear índices originais
    
    for i in range(n_steps, len(scaled_data)):
        # Input: n_steps valores anteriores de Vazao_BBR
        X.append(scaled_data[i-n_steps:i, 0])
        # Target: próximo valor de Vazao_BBR
        y.append(scaled_data[i, 0])
        indices.append(i)
    
    # Reformatar X para ter shape (samples, timesteps, features)
    # Para univariado: features = 1
    X = np.array(X)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    return np.array(X), np.array(y), scaler, indices


def prepare_non_imputed_sequences(data, n_steps, target_col='Vazao_BBR'):
    """
    Prepara sequências para dados não imputados, removendo valores -1
    """
    # Filtrar dados: remover linhas onde Vazao_BBR = -1
    valid_data = data[data[target_col] != -1].copy()
    
    if len(valid_data) == 0:
        return np.array([]), np.array([]), None, []
    
    # Usar apenas a coluna Vazao_BBR
    vazao_data = valid_data[[target_col]].values
    
    # Normalizar dados
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(vazao_data)
    
    X, y = [], []
    indices = []  # Para rastrear índices originais (após filtragem)
    
    for i in range(n_steps, len(scaled_data)):
        # Input: n_steps valores anteriores de Vazao_BBR
        X.append(scaled_data[i-n_steps:i, 0])
        # Target: próximo valor de Vazao_BBR
        y.append(scaled_data[i, 0])
        indices.append(i)  # Índices relativos ao valid_data
    
    if len(X) == 0:
        return np.array([]), np.array([]), scaler, []
    
    # Reformatar X para ter shape (samples, timesteps, features)
    X = np.array(X)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    return np.array(X), np.array(y), scaler, indices


def create_lstm_model(input_shape, units, dropout, learning_rate):
    """
    Cria modelo LSTM para predição univariada
    """
    model = Sequential([
        LSTM(units, activation='relu', return_sequences=True, input_shape=input_shape),
        Dropout(dropout),
        LSTM(units // 2, activation='relu'),
        Dropout(dropout),
        Dense(units // 4, activation='relu'),
        Dense(1)  # Predição de 1 valor (Vazao_BBR no próximo timestep)
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model


def create_gru_model(input_shape, units, dropout, learning_rate):
    """
    Cria modelo GRU para predição univariada
    """
    model = Sequential([
        GRU(units, activation='relu', return_sequences=True, input_shape=input_shape),
        Dropout(dropout),
        GRU(units // 2, activation='relu'),
        Dropout(dropout),
        Dense(units // 4, activation='relu'),
        Dense(1)  # Predição de 1 valor (Vazao_BBR no próximo timestep)
    ])
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])
    
    return model


def calculate_rmse(y_true, y_pred):
    """
    Calcula RMSE entre valores reais e preditos
    """
    if len(y_true) == 0 or len(y_pred) == 0:
        return float('inf')
    
    return np.sqrt(mean_squared_error(y_true, y_pred))


def train_and_evaluate(X_train, y_train, X_test, y_test, model_type, params):
    """
    Treina e avalia um modelo
    """
    if len(X_train) == 0 or len(X_test) == 0:
        return float('inf'), None
    
    input_shape = (X_train.shape[1], X_train.shape[2])
    
    if model_type == 'lstm':
        model = create_lstm_model(input_shape, params['units'], params['dropout'], params['learning_rate'])
    else:
        model = create_gru_model(input_shape, params['units'], params['dropout'], params['learning_rate'])
    
    early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    model.fit(
        X_train, y_train,
        epochs=params['epochs'],
        batch_size=params['batch_size'],
        validation_split=0.2,
        callbacks=[early_stop],
        verbose=0
    )
    
    predictions = model.predict(X_test, verbose=0).flatten()
    
    # Calcular RMSE
    rmse = calculate_rmse(y_test, predictions)
    
    return rmse, model


def find_best_universal_params(imputed_file, non_imputed_file, n_steps, param_grid):
    """
    Encontra os melhores parâmetros universais usando dados imputados como referência
    """
    print(f"Encontrando melhores parâmetros universais...")
    
    # Carregar dados imputados (usar como referência principal)
    df_imputed = pd.read_csv(imputed_file)
    df_imputed['Data'] = pd.to_datetime(df_imputed['Data'])
    df_imputed = df_imputed.sort_values('Data').reset_index(drop=True)
    
    # Preparar sequências dos dados imputados
    X_imp, y_imp, scaler_imp, indices_imp = prepare_univariate_sequences(df_imputed, n_steps)
    
    if len(X_imp) == 0:
        print("Dados imputados insuficientes para treinamento")
        return None
    
    # Split treino/teste para dados imputados
    split_idx = int(len(X_imp) * 0.8)
    X_train_imp, X_test_imp = X_imp[:split_idx], X_imp[split_idx:]
    y_train_imp, y_test_imp = y_imp[:split_idx], y_imp[split_idx:]
    
    # Testar combinações de parâmetros para ambos os modelos
    best_combination = None
    best_avg_rmse = float('inf')
    
    for units in param_grid['units']:
        for dropout in param_grid['dropout']:
            for lr in param_grid['learning_rate']:
                for batch_size in param_grid['batch_size']:
                    params = {
                        'units': units,
                        'dropout': dropout,
                        'learning_rate': lr,
                        'batch_size': batch_size,
                        'epochs': param_grid['epochs']
                    }
                    
                    try:
                        # Testar LSTM com dados imputados
                        lstm_rmse_imp, _ = train_and_evaluate(
                            X_train_imp, y_train_imp, X_test_imp, y_test_imp, 'lstm', params
                        )
                        
                        # Testar GRU com dados imputados
                        gru_rmse_imp, _ = train_and_evaluate(
                            X_train_imp, y_train_imp, X_test_imp, y_test_imp, 'gru', params
                        )
                        
                        avg_rmse = (lstm_rmse_imp + gru_rmse_imp) / 2
                        
                        if avg_rmse < best_avg_rmse:
                            best_avg_rmse = avg_rmse
                            best_combination = params
                            print(f"  Novos melhores parâmetros: RMSE médio={avg_rmse:.4f}, params={params}")
                            
                    except Exception as e:
                        print(f"  Erro com params {params}: {e}")
                        continue
    
    if best_combination:
        print(f"Melhores parâmetros universais encontrados: {best_combination}")
    else:
        print("Não foi possível encontrar parâmetros adequados")
    
    return best_combination


def evaluate_single_dataset(file_path, n_steps, model_type, best_params, is_imputed=True):
    """
    Avalia um modelo em um único dataset (com ou sem imputação)
    """
    # Carregar dados
    df = pd.read_csv(file_path)
    df['Data'] = pd.to_datetime(df['Data'])
    df = df.sort_values('Data').reset_index(drop=True)
    
    # Preparar sequências baseado no tipo de dados
    if is_imputed:
        X, y, scaler, indices = prepare_univariate_sequences(df, n_steps)
    else:
        X, y, scaler, indices = prepare_non_imputed_sequences(df, n_steps)
    
    if len(X) == 0:
        print(f"  Dados insuficientes após preparação")
        return float('inf'), best_params, None, indices
    
    # Split treino/teste (80/20)
    split_idx = int(len(X) * 0.8)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # Treinar e avaliar com parâmetros fixos
    rmse, model = train_and_evaluate(X_train, y_train, X_test, y_test, model_type, best_params)
    
    # Fazer predições para todo o dataset se possível
    predictions = None
    if len(X) > 0:
        # Treinar modelo final com todos os dados de treino
        input_shape = (X_train.shape[1], X_train.shape[2])
        if model_type == 'lstm':
            final_model = create_lstm_model(input_shape, best_params['units'], 
                                          best_params['dropout'], best_params['learning_rate'])
        else:
            final_model = create_gru_model(input_shape, best_params['units'], 
                                         best_params['dropout'], best_params['learning_rate'])
        
        final_model.fit(X_train, y_train, epochs=best_params['epochs'], 
                       batch_size=best_params['batch_size'], validation_split=0.2,
                       callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)], 
                       verbose=0)
        
        # Predições em todo o dataset
        all_predictions = final_model.predict(X, verbose=0).flatten()
        
        # Desnormalizar predições
        predictions_denorm = scaler.inverse_transform(all_predictions.reshape(-1, 1)).flatten()
        predictions = predictions_denorm
    
    return rmse, best_params, predictions, indices


def compare_imputation_vs_non_imputation(imputed_folder, non_imputed_folder, n_steps=12):
    """
    Compara performance de LSTM e GRU com e sem imputação
    """
    # Configuração de hiperparâmetros
    param_grid = {
        'units': [32, 64, 128],
        'dropout': [0.1, 0.2, 0.3],
        'learning_rate': [0.001, 0.0005],
        'batch_size': [16, 32],
        'epochs': 100
    }
    
    # Encontrar arquivos correspondentes
    matched_files = find_matching_files(imputed_folder, non_imputed_folder)
    
    if not matched_files:
        print("Nenhum par de arquivos correspondente encontrado!")
        return pd.DataFrame()
    
    results = []
    
    for link, files in matched_files.items():
        print(f"\n{'='*60}")
        print(f"Processando link: {link}")
        print(f"{'='*60}")
        
        # Encontrar melhores parâmetros universais para este link
        best_params = find_best_universal_params(
            files['imputed'], files['non_imputed'], n_steps, param_grid
        )
        
        if not best_params:
            print(f"  Não foi possível encontrar parâmetros para {link}. Pulando...")
            continue
        
        link_results = {}
        
        # Avaliar cada combinação: modelo × tipo de dados
        for model_type in ['lstm', 'gru']:
            print(f"\n  Avaliando {model_type.upper()}...")
            
            # Com imputação
            print(f"    Com imputação...")
            rmse_imputed, _, preds_imp, indices_imp = evaluate_single_dataset(
                files['imputed'], n_steps, model_type, best_params, is_imputed=True
            )
            
            # Sem imputação
            print(f"    Sem imputação...")
            rmse_non_imputed, _, preds_non_imp, indices_non_imp = evaluate_single_dataset(
                files['non_imputed'], n_steps, model_type, best_params, is_imputed=False
            )
            
            link_results[model_type] = {
                'rmse_imputed': rmse_imputed,
                'rmse_non_imputed': rmse_non_imputed,
                'predictions_imputed': preds_imp,
                'predictions_non_imputed': preds_non_imp,
                'indices_imputed': indices_imp,
                'indices_non_imputed': indices_non_imp
            }
            
            print(f"    {model_type.upper()} - Com imputação: {rmse_imputed:.4f}")
            print(f"    {model_type.upper()} - Sem imputação: {rmse_non_imputed:.4f}")
            
            # Calcular melhoria da imputação
            if rmse_non_imputed != float('inf') and rmse_imputed != float('inf'):
                improvement = ((rmse_non_imputed - rmse_imputed) / rmse_non_imputed) * 100
            else:
                improvement = 0
            
            # Salvar resultado
            results.append({
                'Link': link,
                'Model': model_type.upper(),
                'RMSE_Com_Imputacao': rmse_imputed,
                'RMSE_Sem_Imputacao': rmse_non_imputed,
                'Melhoria_%': improvement,
                'Params': str(best_params)
            })
        
        # Salvar predições para análise posterior
        save_predictions(link, files['imputed'], link_results)
    
    return pd.DataFrame(results)


def save_predictions(link, original_file, results):
    """
    Salva predições em arquivos CSV para análise
    """
    # Carregar dados originais
    df_original = pd.read_csv(original_file)
    df_original['Data'] = pd.to_datetime(df_original['Data'])
    df_original = df_original.sort_values('Data').reset_index(drop=True)
    
    # Criar dataframe com predições
    df_pred = df_original.copy()
    
    for model_type in ['lstm', 'gru']:
        # Predições com imputação
        pred_imp = results[model_type]['predictions_imputed']
        indices_imp = results[model_type]['indices_imputed']
        
        if pred_imp is not None and len(pred_imp) > 0:
            df_pred[f'Predict_{model_type.upper()}_Imputed'] = np.nan
            df_pred.loc[indices_imp, f'Predict_{model_type.upper()}_Imputed'] = pred_imp
    
    # Salvar arquivo
    output_file = f"comparison_predictions_{link}.csv"
    df_pred.to_csv(output_file, index=False)
    print(f"  Predições salvas em: {output_file}")


def main(imputed_folder, non_imputed_folder, n_steps=12):
    """
    Função principal para executar comparação
    """
    print(f"\n{'#'*80}")
    print(f"# COMPARAÇÃO: COM vs SEM IMPUTAÇÃO")
    print(f"# Modelos: LSTM e GRU")
    print(f"# Dados imputados: {imputed_folder}")
    print(f"# Dados não imputados: {non_imputed_folder}")
    print(f"# {n_steps} timesteps passados para prever 1 passo à frente")
    print(f"{'#'*80}\n")
    
    # Executar comparação
    results_df = compare_imputation_vs_non_imputation(
        imputed_folder, non_imputed_folder, n_steps
    )
    
    if results_df.empty:
        print("Nenhum resultado foi gerado!")
        return results_df
    
    # Filtrar casos válidos
    valid_results = results_df[
        (results_df['RMSE_Com_Imputacao'] != float('inf')) & 
        (results_df['RMSE_Sem_Imputacao'] != float('inf'))
    ].copy()
    
    # Salvar resultados
    output_file = 'resultados_comparacao_imputacao.csv'
    valid_results.to_csv(output_file, index=False)
    print(f"\nResultados salvos em: {output_file}")
    
    # Mostrar resumo
    print("\n" + "="*80)
    print("RESUMO DA COMPARAÇÃO")
    print("="*80)
    
    if len(valid_results) > 0:
        print(valid_results.to_string(index=False))
        
        # Estatísticas
        print(f"\n{'='*80}")
        print("ESTATÍSTICAS GERAIS")
        print(f"{'='*80}")
        print(f"Total de comparações válidas: {len(valid_results)}")
        print(f"Melhoria média da imputação: {valid_results['Melhoria_%'].mean():.2f}%")
        print(f"Melhoria mediana da imputação: {valid_results['Melhoria_%'].median():.2f}%")
        print(f"Casos com melhoria positiva: {len(valid_results[valid_results['Melhoria_%'] > 0])}")
        print(f"Casos com piora: {len(valid_results[valid_results['Melhoria_%'] < 0])}")
        
        # Por modelo
        for model in ['LSTM', 'GRU']:
            model_results = valid_results[valid_results['Model'] == model]
            if len(model_results) > 0:
                print(f"\n{model}:")
                print(f"  Melhoria média: {model_results['Melhoria_%'].mean():.2f}%")
                print(f"  Melhores casos: {len(model_results[model_results['Melhoria_%'] > 0])}")
    else:
        print("Nenhum resultado válido para análise")
    
    return valid_results


# Exemplo de uso:
if __name__ == "__main__":
    # Substitua pelos caminhos das suas pastas
    IMPUTED_FOLDER = "../../datasets/multivariada-imputed-selecionados"
    NON_IMPUTED_FOLDER = "../../datasets/multivariada-post-process"  # Ajuste este caminho
    
    # Executar comparação
    results = main(IMPUTED_FOLDER, NON_IMPUTED_FOLDER, n_steps=12)


################################################################################
# COMPARAÇÃO: COM vs SEM IMPUTAÇÃO
# Modelos: LSTM e GRU
# Dados imputados: ../../datasets/multivariada-imputed-selecionados
# Dados não imputados: ../../datasets/multivariada-post-process
# 12 timesteps passados para prever 1 passo à frente
################################################################################

Encontrados 406 pares de arquivos correspondentes

Processando link: ac-ap
Encontrando melhores parâmetros universais...
  Novos melhores parâmetros: RMSE médio=0.1361, params={'units': 32, 'dropout': 0.1, 'learning_rate': 0.001, 'batch_size': 16, 'epochs': 100}
  Novos melhores parâmetros: RMSE médio=0.1357, params={'units': 32, 'dropout': 0.2, 'learning_rate': 0.001, 'batch_size': 16, 'epochs': 100}
  Novos melhores parâmetros: RMSE médio=0.1353, params={'units': 32, 'dropout': 0.2, 'learning_rate': 0.001, 'batch_size': 32, 'epochs': 100}
  Novos melhores parâmetros: RMSE médio=0.1352,